### Script for fastqc

After downloading all the fastq files from Nanuq server, they were transfered to calculon `/scratch/chengar2/geneD` batch by batch to avoid overcrowding the storage.

First task is to evaluate the integrity of the files and check for error/corruption during file transfer using `FastQC`.

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=fastqc
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=8gb
#SBATCH --output=/home/chengar2/geneD/fastqc_%A.out
#SBATCH --error=/home/chengar2/geneD/fastqc_%A.err

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load fastqc/0.11.8

fq=/scratch/chengar2/geneD/*.fastq.gz

fastqc $fq

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

### Script for trimmomatic (last update 20191221)

I made specific adapter set to match the adapter sets provided by Nanuq. The file is saved as `/scratch/chengar2/geneD/adapter.fa`

    >Read1
    AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC
    >Read2
    AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
    >Read1_rc
    GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT
    >Read2_rc
    ACACTCTTTCCCTACACGACGCTCTTCCGATCT

I couldn't load trimmomatic like I loaded FastQC for some reason... Ended up having to dig up the path to the `.jar` files to use trimmomatic. 

These trimmed files are __NOT__ meant to be deposited into EMBI repository since they don't accept quality-trimmed fastq (as of SOX2 paper submission; rules could have changed).

I hard coded the name and the path of the `fastq` files in this section; would not recommend this method. See other sections below for using for loops.

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=trim_test
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=8gb
#SBATCH --output=/home/chengar2/geneD/trim_%A.out
#SBATCH --error=/home/chengar2/geneD/trim_%A.err

start=`date +%s`
echo $HOSTNAME
date

module load trimmomatic/0.39

outpath=/scratch/chengar2/geneD/qual_trim/
trim_p1=.trim.paired.1.fq.gz
trim_u1=.trim.UNpaired.1.fq.gz
trim_p2=.trim.paired.2.fq.gz
trim_u2=.trim.UNpaired.2.fq.gz

fq1=/scratch/chengar2/geneD/NS.1238.004.NEBNext_dual_i7_F2---NEBNext_dual_i5_F2.KO3-LP3_R1.fastq.gz
fq2=/scratch/chengar2/geneD/NS.1238.004.NEBNext_dual_i7_F2---NEBNext_dual_i5_F2.KO3-LP3_R2.fastq.gz

java -jar /cm/shared/apps/trimmomatic/0.39/trimmomatic-0.39.jar PE \
-threads 4 \
-phred33 \
$fq1 $fq2 \
$outpath${fq1:76:10}$trim_p1 \
$outpath${fq1:76:10}$trim_u1 \
$outpath${fq2:76:10}$trim_p2 \
$outpath${fq2:76:10}$trim_u2 \
ILLUMINACLIP:/scratch/chengar2/geneD/adapter.fa:2:30:10 \
LEADING:3 \
TRAILING:3 \
SLIDINGWINDOW:4:15 \
MINLEN:36

end=`date +%s`
runtime=$((end-start))
date
echo $runtime

### Download genome files:

GTF gene annotation and genome sequence is downloaded from here: https://useast.ensembl.org/info/data/ftp/index.html

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=trim_test
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --mem=5gb
#SBATCH --output=/home/chengar2/geneD/wget_%A.out
#SBATCH --error=/home/chengar2/geneD/wget_%A.err

start=`date +%s`
echo $HOSTNAME
date

#GTF gene annotation
wget -P /scratch/chengar2/geneD/gtf ftp://ftp.ensembl.org/pub/release-98/gtf/mus_musculus/*

#genome sequence
wget -P /scratch/chengar2/geneD/dna ftp://ftp.ensembl.org/pub/release-98/fasta/mus_musculus/dna/*

end=`date +%s`
runtime=$((end-start))
date
echo $runtime

### Building genome index ###

After unzipping the toplevel.fa.gz, I built a large index using bowtie2 with the code below. Output files are in `/scratch/chengar2/geneD/dna/`

> #### Primary assembly vs Top level?
"The short answer is to choose the "Primary assembly" (i.e. Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa.gz) as it does not contain the haplotype information. The top level file contains additional sequences that are relatively common variants to the reference. Most mappers available now don't specifically handle these haplo sequences and as such they will appear as simply another contig, therefore complicating the alignment. Perhaps in future, mappers might be better able to handle these hypervariable regions."  
--http://genomespot.blogspot.com/2015/06/mapping-ngs-data-which-genome-version.html

Code below is for building the index with **Bowtie2**

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=index
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=24
#SBATCH --mem=24gb
#SBATCH --output=/home/chengar2/geneD/index_%A.out
#SBATCH --error=/home/chengar2/geneD/index_%A.err

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load bowtie2/2.3.4.3

gunzip -c /scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.toplevel.fa.gz >/scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.toplevel.fa

bowtie2-build --large-index --threads 24 /scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.toplevel.fa \
/scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.toplevel

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

Code below is for building the index with **Bowtie2**

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=index-pri
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=15
#SBATCH --mem=40gb
#SBATCH --output=/home/chengar2/geneD/pri-assem_%A.out
#SBATCH --error=/home/chengar2/geneD/pri-assem_%A.err

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load bowtie2/2.3.4.3

gunzip -c /scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.primary_assembly.fa.gz >/scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.primary_assembly.fa

bowtie2-build --large-index --threads 24 /scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.primary_assembly.fa \
/scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.primary_assembly

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

### Building genome index v2

`hisat2` is supposedly an improved version of `bowtie2` that is better at identifying reads that span over an intron. To prepare alignment with `hisat2`, I indexed the mouse reference genome again with `hisat2-build`, in two different ways.

Code below is for building the index with **hisat2**

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=index
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=24
#SBATCH --mem=24gb
#SBATCH --output=/home/chengar2/geneD/index_%A.out
#SBATCH --error=/home/chengar2/geneD/index_%A.err

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load hisat2/2.1.0

hisat2-build --large-index --threads 24 /scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.toplevel.fa \
/scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.toplevel

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

Code below is for building the index with **hisat2**

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=index
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=15
#SBATCH --mem=40gb
#SBATCH --output=/home/chengar2/geneD/index_%A.out
#SBATCH --error=/home/chengar2/geneD/index_%A.err

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load hisat2/2.1.0

hisat2-build --large-index --threads 15 /scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.primary_assembly.fa \
/scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.primary_assembly

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

---
### Alignment

There is countless modules (bwa, bowtie, tophat, hisat, salmon, STAR etc) made for aligning RNA-seq data to a reference genome and each has its own strength and drawback.

I used both `bowtie2` and `hisat2` for alignment here and explored their performance in handling transcriptomic data.

P.S. `hisat2` runs a lot faster than `bowtie2` (2 vs ~4+ hours per files) with equal resources


Code below is for alignment with **`Bowtie2`** using **`toplevel`** index

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=align
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=18gb
#SBATCH --output=/home/chengar2/geneD/align_%A.out
#SBATCH --error=/home/chengar2/geneD/align_%A.err

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load bowtie2/2.3.4.3

for fq in /scratch/chengar2/geneD/qual_trim/*.trim.paired.1.fq.gz
do
    echo "running analysis on ${fq:34:7}"
    name=${fq:34:7}
    filebase=${fq:0:41}
    filebase1=${fq:0:44}
    filebase2=${fq:0:43}2
    bowtie2 --un-gz $filebase.unpaired.unmapped.reads.fq \
    --un-conc-gz $filebase.paired.discordant.reads.fq \
    --met-file $filebase.metrics --rg-lsid $name \
    -x /scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.toplevel \
    -1 $filebase1.trim.paired.1.fq.gz \
    -2 $filebase2.trim.paired.2.fq.gz \
    -U $filebase1.trim.UNpaired.1.fq.gz,$filebase2.trim.UNpaired.2.fq.gz \
    -S $filebase.sam 2>$filebase.stderr.txt
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

Code below is for alignment with **`Bowtie2`** using **`primary assembly`** index

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=bowtie2_pri
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=24gb
#SBATCH --output=/home/chengar2/geneD/bowtie2_pri_%A.out
#SBATCH --error=/home/chengar2/geneD/bowtie2_pri_%A.err

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load bowtie2/2.3.4.3

for fq in /scratch/chengar2/geneD/hisat2/*.trim.paired.1.fq.gz
do
    filebase=${fq:0:38}
    filebase1=${fq:0:41}
    filebase2=${fq:0:40}2
    name=${fq:31:7}
    echo "running analysis for $name on $(date)"
    bowtie2 --un-gz $filebase.pri.bow2.unpaired.unmapped.reads.fq \
    --un-conc-gz $filebase.pri.bow2.paired.discordant.reads.fq \
    --met-file $filebase.pri.bow2.metrics --rg-id geneD_$name --rg LB:lib1 --rg PL:illumina --rg SM:$name --rg PU:unit1 \
    -x /scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.primary_assembly\
    -1 $filebase1.trim.paired.1.fq.gz \
    -2 $filebase2.trim.paired.2.fq.gz \
    -U $filebase1.trim.UNpaired.1.fq.gz,$filebase2.trim.UNpaired.2.fq.gz \
    -S $filebase.pri.bow2.sam 2>$filebase.pri.bow2.stderr.txt
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

### Alignment v2 - hisat2

I ran a test with `hisat2` to evaluate the performance of the program. Turns out `hisat2` performance is far better than `bowtie2` using `toplevel` index (98.51% vs 84.75%). ~14% might seems inconsequential however we are dealing with 43 millions reads, 14% is roughly 6 millions read!! In addition if we focus on number of reads that "aligned concordantly exactly 1 time", we see a huge % difference (91.17% vs 52.7%); along the same line there is 10 times less reads that "aligned concordantly 0 times" (1.3mil vs 14mil) when I use `hisat2`. 

**Sample: KO1-DD1; `hisat2` using `toplevel` index**  ---------------------------------------------------------------------  
43922113 reads; of these:

    >42760471 (97.36%) were paired; of these:  
        >>1384617 (3.24%) aligned concordantly 0 times  
        >>38984529 (91.17%) aligned concordantly exactly 1 time  
        >>2391325 (5.59%) aligned concordantly >1 times  
    
        >>1384617 pairs aligned concordantly 0 times; of these:  
            >>>345607 (24.96%) aligned discordantly 1 time
        
        >>1039010 pairs aligned 0 times concordantly or discordantly; of these:  
            >>>2078020 mates make up the pairs; of these:  
                >>>>1106485 (53.25%) aligned 0 times  
                >>>>726118 (34.94%) aligned exactly 1 time  
                >>>>245417 (11.81%) aligned >1 times  
    >1161642 (2.64%) were unpaired; of these:  
        >>184642 (15.89%) aligned 0 times  
        >>902367 (77.68%) aligned exactly 1 time  
        >>74633 (6.42%) aligned >1 times  
**98.51% overall alignment rate**

---

**Sample: KO1-DD1; `bowtie2` using `toplevel` index**  -----------------------------------------------------------------------  
43922113 reads; of these:
  
    >42760471 (97.36%) were paired; of these:
        >>14473868 (33.85%) aligned concordantly 0 times
        >>22533086 (52.70%) aligned concordantly exactly 1 time
        >>5753517 (13.46%) aligned concordantly >1 times
    
        >>14473868 pairs aligned concordantly 0 times; of these:
          >>>3701127 (25.57%) aligned discordantly 1 time
    
        >>10772741 pairs aligned 0 times concordantly or discordantly; of these:
          >>>21545482 mates make up the pairs; of these:
            >>>>12959303 (60.15%) aligned 0 times
            >>>>7400580 (34.35%) aligned exactly 1 time
            >>>>1185599 (5.50%) aligned >1 times
    >1161642 (2.64%) were unpaired; of these:
        >>260480 (22.42%) aligned 0 times
        >>724215 (62.34%) aligned exactly 1 time
        >>176947 (15.23%) aligned >1 times
**84.75% overall alignment rate**

:-----------------------------------------------------------------------------------------------------------------------------  
On another note, when I switched the index file to  `primary assembly` instead of `toplevel`, the alignment % interesting remained practically the same (0.2% lower). However it is also worth noting that more reads aligned concordantly exactly once (92.67% vs 91.17%) while less reads aligned concordantly >1 times (4.08% vs 5.59%) when I switched to `primary assembly`. 

Since `hisat2` runs pretty fast, we can align all of our `fastq` to both `primary assembly` and `toplevel` index to allow a better comparison. 

**Sample: KO1-DD1; `hisat2` using `primary assembly` index**------------------------------------------------------------    
43922113 reads; of these:
     
    >42760471 (97.36%) were paired; of these:
        >>1391064 (3.25%) aligned concordantly 0 times
        >>39625929 (92.67%) aligned concordantly exactly 1 time
        >>1743478 (4.08%) aligned concordantly >1 times
    
        >>1391064 pairs aligned concordantly 0 times; of these:
          >>>352189 (25.32%) aligned discordantly 1 time
    
        >>1038875 pairs aligned 0 times concordantly or discordantly; of these:
          >>>2077750 mates make up the pairs; of these:
            >>>>1122062 (54.00%) aligned 0 times
            >>>>735210 (35.38%) aligned exactly 1 time
            >>>>220478 (10.61%) aligned >1 times
    >1161642 (2.64%) were unpaired; of these:
        >>184979 (15.92%) aligned 0 times
        >>918263 (79.05%) aligned exactly 1 time
        >>58400 (5.03%) aligned >1 times
**98.49% overall alignment rate**

**Sample: KO1-DD1; `bowtie2` using `primary assembly` index**------------------------------------------------------------    
43922113 reads; of these:

    >42760471 (97.36%) were paired; of these:
        >>14479664 (33.86%) aligned concordantly 0 times
        >>22962797 (53.70%) aligned concordantly exactly 1 time
        >>5318010 (12.44%) aligned concordantly >1 times
    
        >>14479664 pairs aligned concordantly 0 times; of these:
          >>>3764337 (26.00%) aligned discordantly 1 time

        >>10715327 pairs aligned 0 times concordantly or discordantly; of these:
          >>>21430654 mates make up the pairs; of these:
            >>>>12975281 (60.55%) aligned 0 times
            >>>>7510451 (35.05%) aligned exactly 1 time
            >>>>944922 (4.41%) aligned >1 times
    >1161642 (2.64%) were unpaired; of these:
        >>260830 (22.45%) aligned 0 times
        >>738323 (63.56%) aligned exactly 1 time
        >>162489 (13.99%) aligned >1 times
**84.73% overall alignment rate**



Code below is for alignment with **`hisat2`** using **`toplevel`** index

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=hisat2
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=18gb
#SBATCH --output=/home/chengar2/geneD/hisat2-align_%A.out
#SBATCH --error=/home/chengar2/geneD/hisat2-align_%A.err

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load hisat2/2.1.0

for fq in /scratch/chengar2/geneD/hisat2/*.trim.paired.1.fq.gz
do
    filebase=${fq:0:38}
    filebase1=${fq:0:41}
    filebase2=${fq:0:40}2
    name=${fq:31:7}
    echo "running analysis for $name on $(date)"
    hisat2 --un-gz $filebase.hisat2.unpaired.unmapped.reads.fq \
    --un-conc-gz $filebase.hisat2.paired.discordant.reads.fq \
    --met-file $filebase.hisat2.metrics --rg-id geneD_$name --rg LB:lib1 --rg PL:illumina --rg SM:$name --rg PU:unit1 \
    -x /scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.toplevel \
    -1 $filebase1.trim.paired.1.fq.gz \
    -2 $filebase2.trim.paired.2.fq.gz \
    -U $filebase1.trim.UNpaired.1.fq.gz,$filebase2.trim.UNpaired.2.fq.gz \
    -S $filebase.hisat2.sam 2>$filebase.hisat2.stderr.txt
    echo "analysis complete for $name on $(date)"
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

Code below is for alignment with **`hisat2`** using **`primary assembly`** index

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=hisat2
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=18gb
#SBATCH --output=/home/chengar2/geneD/hisat2_pri-align_%A.out
#SBATCH --error=/home/chengar2/geneD/hisat2_pri-align_%A.err

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load hisat2/2.1.0

for fq in /scratch/chengar2/geneD/hisat2/*.trim.paired.1.fq.gz
do
    filebase=${fq:0:38}
    filebase1=${fq:0:41}
    filebase2=${fq:0:40}2
    name=${fq:31:7}
    echo "running analysis for $name on $(date)"
    hisat2 --un-gz $filebase.pri.hisat2.unpaired.unmapped.reads.fq \
    --un-conc-gz $filebase.pri.hisat2.paired.discordant.reads.fq \
    --met-file $filebase.pri.hisat2.metrics --rg-id geneD_$name --rg LB:lib1 --rg PL:illumina --rg SM:$name --rg PU:unit1 \
    -x /scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.primary_assembly \
    -1 $filebase1.trim.paired.1.fq.gz \
    -2 $filebase2.trim.paired.2.fq.gz \
    -U $filebase1.trim.UNpaired.1.fq.gz,$filebase2.trim.UNpaired.2.fq.gz \
    -S $filebase.pri.hisat2.sam 2>$filebase.pri.hisat2.stderr.txt
    echo "analysis complete for $name on $(date)"
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

### Alignment v3 - Strandness


Assuming Genome Quebec is still using the same dUTP library preparation for the SOX2 paper, our RNA-seq should be strand-specific and the library strand should be first strand:

>"cDNA synthesis was achieved with the NEBNext RNA First Strand Synthesis and `NEBNext Ultra Directional RNA Second Strand Synthesis Modules (New England BioLabs)`. The remaining steps of library preparation were done using the NEBNext Ultra II DNA Library Prep Kit for Illumina (New England BioLabs). Adapters and PCR primers were purchased from New England BioLabs. " -- Genome Quebec protocol for SOX2 paper
>>NEB protocol: https://international.neb.com/products/e7550-nebnext-ultra-directional-rna-second-strand-synthesis-module#Product%20Information
__TL;DR__ dUTP was incorporated during the second strand synthesis and digested after adapter ligation, leaving only the first cDNA strand (reverse complement of the mRNA) to form the library. Thus the library strand is the first strand by EBI/ENA definition

>Overview of strand-specific library preparation: https://github.com/igordot/genomics/blob/master/notes/rna-seq-strand.md

>More discussion on strandness with a great representative image at the bottom of the page https://www.biostars.org/p/98756/

And we can confirm this by visualizing one of our indexed `bam` files with IGV. Example images are from `filtered_WT1-DD1.bam` using `hisat2` and `primary_assembly` for alignment.



### Example 1: Gene is on negative strand

![Stox2](gene_on_neg.png)

**Observation** 
- There is a lot more **red** alignments than **blue** alignments. 
- There is actually a lot more **red** alignments when you scroll further down. There is so many **red** I can't capture all of them in one frame.
- Blue means read 1 of the pair is aligned to negative strand; red means read 1 of the pair is aligned to the positive strand
- Reference genome shows this gene `Stox2` is encoded on the negative strand (solid blue bar at the bottom with "<" showing directionality)

**Interpretation**  
- No. of Red alignments >> No. of Blue  alignments -> this is definitely stranded. (Not surprising; if you go to Nanuq page where `fastq` can be downloaded, there is a column called `"library type"` with the value of `"mRNA stranded"`)
- **Most** read 2 of the pair aligned to the negative strand, which make sense because if the library strand is the reverse complement of the mRNA -> read 1 will align on the template strand; read 2 will align on the coding strand.

**Summary**
- R2 aligned on negative strand while ref. genome shows the gene is encoded by the negative strand == first strand is library strand


### Example 2: Gene is on positive strand

![Sox2](gene_on_pos.png)

**Observation** 
- There is a lot more **blue** alignments than **red** alignments. 
- There is actually a lot more **blue** alignments when you scroll further up. There is so many **blue** I can't capture all of them in one frame.
- Blue means read 1 of the pair is aligned to negative strand; red means read 1 of the pair is aligned to the positive strand
- Reference genome shows this gene `Sox2` is encoded on the positive strand (solid blue bar at the bottom with ">" showing directionality)

**Interpretation**  
- No. of Blue alignments >> No. of Red  alignments -> this is definitely stranded. (Once again not surprising)
- **Most** read 2 of the pair aligned to the positive strand, which make sense because if the library strand is the reverse complement of the mRNA -> read 1 will align on the template strand; read 2 will align on the coding strand.

**Summary**
- R2 aligned on positive strand while ref. genome shows the gene is encoded by the positive strand == first strand is library strand


### Would this change the way we code? 

There is the option `--rna-strandness` for user to specify strandness of the library in `hisat2`, which means in our case we can type `--rna-strandness RF` to let `hisat2` know our library strand is the first strand. 

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=strand
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=24gb
#SBATCH --output=/home/chengar2/geneD/strand_%A.out
#SBATCH --error=/home/chengar2/geneD/strand_%A.err

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load hisat2/2.1.0

for fq in /scratch/chengar2/geneD/hisat2/*.trim.paired.1.fq.gz
do
    filebase=${fq:0:38}
    filebase1=${fq:0:41}
    filebase2=${fq:0:40}2
    name=${fq:31:7}
    echo "running analysis for $name on $(date)"
    hisat2 --rna-strandness RF -p 6 \
    --un-gz $filebase.pri.hisat2.unpaired.unmapped.reads.fq \
    --un-conc-gz $filebase.pri.hisat2.paired.discordant.reads.fq \
    --met-file $filebase.pri.hisat2.metrics --rg-id geneD_$name --rg LB:lib1 --rg PL:illumina --rg SM:$name --rg PU:unit1 \
    -x /scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.primary_assembly \
    -1 $filebase1.trim.paired.1.fq.gz \
    -2 $filebase2.trim.paired.2.fq.gz \
    -U $filebase1.trim.UNpaired.1.fq.gz,$filebase2.trim.UNpaired.2.fq.gz \
    -S $filebase.pri.hisat2.sam 2>$filebase.pri.hisat2.stderr.txt
    echo "analysis complete for $name on $(date)"
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

### Quality check with Picard
To make sure the alignments are done properly, we can use picard ValidateSamFile function to check for anomalies in the Sam files.

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=picard
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=24gb
#SBATCH --output=/home/chengar2/geneD/picard_%A.out
#SBATCH --error=/home/chengar2/geneD/picard_%A.err


start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load picard/2.13

for sam in /scratch/chengar2/geneD/qual_trim/*.sam
do
    extract=$(echo $sam | awk -F"." '{print $1}')
    name=$(echo $extract | awk -F"/" '{print $NF}')
    echo "running analysis for $name on $(date)"
    java -jar /cm/shared/apps/picard/2.13/picard.jar ValidateSamFile I=$sam MODE=SUMMARY \
    O=/scratch/chengar2/geneD/bam/$name.txt
    echo "analysis complete for $name on $(date)"
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

### `hisat2` giving us errors!

When we run `picard ValidateSamFile` on the `sam` generated by `hisat2` we encountered a problem.

>ERROR: Record 178, Read name A00977:10:HWCTJDSXX:4:1101:16938:1078, Mate negative strand flag does not match read negative strand flag of mate  
ERROR: Record 616, Read name A00977:10:HWCTJDSXX:4:1101:2528:1235, Mate negative strand flag does not match read negative strand flag of mate  
ERROR: Record 624, Read name A00977:10:HWCTJDSXX:4:1101:6072:1235, Mate negative strand flag does not match read negative strand flag of mate  
ERROR: Record 878, Read name A00977:10:HWCTJDSXX:4:1101:24813:1313, Mate negative strand flag does not match read negative strand flag of mate  
ERROR: Record 904, Read name A00977:10:HWCTJDSXX:4:1101:21875:1329, Mate negative strand flag does not match read negative strand flag of mate  

That is just 5 out of the ~300,000 errors in one `sam` file! They all have the same problem "MISMATCH_FLAG_MATE_NEG_STRAND".

This gave me a huge headache but turns out to be something not too serious. A weird thing with `hisat2` is that it doesn't assign the correct tag/flag to a read when a read's mate in the pair cannot be aligned, in particular when one of them is aligned to the negative strand.

To be more specific, each read gets a specific number to identify their (and their mate's) properties (e.g. read paired? read mapped in proper pair? read unmapped?....), which is encoded in the integer (89) after the read name (A00977:10:HWCTJDSXX:4:1101:20130:2816) **see read pair sample 1**. To know what 89 stands for, one would have to decode it or look it up. Luckily we can use the tool here https://broadinstitute.github.io/picard/explain-flags.html , simply input 89 into the "SAM flag" search box and click "explain", we can see that this read is: 
>"read paired (0x1), mate unmapped (0x8), read reverse strand (0x10), first in pair (0x40)"

The proper flag for its mate should be 165 (press "switch to mate" to find out), which corresponds to:
>"read paired (0x1), read unmapped (0x4), mate reverse strand (0x20), second in pair (0x80)"

However `hisat2` gave its mate 133, which corresponds to:
>"read paired (0x1), read unmapped (0x4), second in pair (0x80)"

...which does not have the "mate reverse strand" part. `hisat2` giving it the "wrong" flag means we have to use `picard FixMateInformation` to correct it but the actual alignment result should be still valid. 

To learn more about the meaning of each field in a SAM file, see https://medium.com/@shilparaopradeep/samtools-guide-learning-how-to-filter-and-manipulate-with-sam-bam-files-2c28b25d29e8

### Read pair sample 1
```bash
>[chengar2@calculon hisat2]$ grep -in A00977:10:HWCTJDSXX:4:1101:20130:2816 KO1-DD1.pri.hisat2.sam
5148:A00977:10:HWCTJDSXX:4:1101:20130:2816      89      9       80139057        60      91M1670N10M     =       80139057        0       AATCAGACTAAACTACGGTGATCAGTCAGCAGACGGTGGGAAGCTGCTTGAAGATGAGCTCATTGACTTCTCAGAGGATCAGGATGACCCGGATGATAGCA      FFFFFFF:F,::,FFFFFFFFFFFFFF:F,F:FF:FFFF,FFFFFF:FFF:FFFFFFF::FF,FFFFFF:FF:FFFFFFFFFFFFFFFFFFFFFF,F:F,F   AS:i:0  XN:i:0  XM:i:0  XO:i:0  XG:i:0  NM:i:0  MD:Z:101        YT:Z:UP RG:Z:geneD_KO1-DD1 XS:A:+  NH:i:1  


>5149:A00977:10:HWCTJDSXX:4:1101:20130:2816      133     9       80139057        0       *       =       80139057        0       TACCGGAGAAACGCATACAGTGTAAAGTGTAGTATGAAAAAAAAAAATAATGCAATCAATGAAAATGAAGAACAAAGTAACGGAGAATAAA        FFFF:FF::FFFFFFFFFFFFFFFFFFF::FFFFFFFFF:FFFF:FFF,F,F,,FFFFF:,FFFF,FFF,:F:,FF,FFF,::,,FF,,,F     YT:Z:UP RG:Z:geneD_KO1-DD1
                                                                                            
```                                                                                            

**I didn't check if all ~300,000 are like this but for the handful that I have, one of the reads in the pair is always unmapped (4th integer after the read name should be 0; it represent MAPQ/mapping quality, unmapped reads have a quality of 0), so for now I presume my conclusion is correct and simply changing the SAM flag will suffice.**

### Read pair sample 2
```bash
[chengar2@calculon hisat2]$ grep -in A00977:10:HWCTJDSXX:4:1101:16685:3458 KO1-DD1.pri.hisat2.sam
6844:A00977:10:HWCTJDSXX:4:1101:16685:3458      89      11      19924932        60      25M73163N76M    =       19924932        0       TGACCGAAGAAACACACCCGGACGATGACAGCTATATTGTGCGTGTCAAGGCTGTGGTTATGACCAGAGATGACTCCAGCGGGGGATGGTTCCCACAGGAA      FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFF   AS:i:-3 XN:i:0  XM:i:0  XO:i:0  XG:i:0  NM:i:0  MD:Z:101        YT:Z:UP RG:Z:geneD_KO1-DD1 XS:A:+  NH:i:1  

6845:A00977:10:HWCTJDSXX:4:1101:16685:3458      133     11      19924932        0       *       =       19924932        0       CGCGGGACAGGCGTCTAGGTGAACAAGAAAATGACCGAAGAAACACACCCGGACGATGACAGCTATATTGTGCGTGTCAAGGCTGTGGTTATGACCAGAGA      FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF   YT:Z:UP RG:Z:geneD_KO1-DD1  

```

### Read pair sample 3

```bash
[chengar2@calculon hisat2]$ grep -in A00977:10:HWCTJDSXX:4:1101:23511:3881 KO1-DD1.pri.hisat2.sam
7981:A00977:10:HWCTJDSXX:4:1101:23511:3881      153     17      23771415        60      6S95M   =       23771415        0       CCCTGTCGACGGCAGCGGGCCCTGCCTCTCCGGCGGCTGAGCCACCCGATCCTTCGTGTGAGTGTCCGCCAACGCAGGCGCCGGCCTAGCAGCCGGGGCTG      FFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF   AS:i:-6 XN:i:0  XM:i:0  XO:i:0  XG:i:0  NM:i:0  MD:Z:95 YT:Z:UP RG:Z:geneD_KO1-DD1NH:i:1
7982:A00977:10:HWCTJDSXX:4:1101:23511:3881      69      17      23771415        0       *       =       23771415        0       CTGGCACCTTCTTCTGGAATGTCTGAAGACAGCCACAGTGTACCTTTAATTCCAGCACTCAGGAGGCAGGGGGTGGAGGAGCTCTGTGAATTTGAGGCCCA      FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFF   YT:Z:UP RG:Z:geneD_KO1-DD1
```

### Read pair sample 4
```bash
[chengar2@calculon hisat2]$ grep -in A00977:10:HWCTJDSXX:4:1101:25527:4554 KO1-DD1.pri.hisat2.sam
10163:A00977:10:HWCTJDSXX:4:1101:25527:4554     153     13      9054582 1       88M     =       9054582 0       TTTTTTTTTTTTTTTTTTTTTTTCCAGAGCTGAGGACCGAACCCAGGGCCTTGCGCTTGCTAGGCAAGCGCTCTACCACTGAGCTAAA        FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFF   AS:i:0  ZS:i:0  XN:i:0  XM:i:0  XO:i:0  XG:i:0  NM:i:0  MD:Z:88 YT:Z:UP RG:Z:geneD_KO1-DD1      NH:i:4
10164:A00977:10:HWCTJDSXX:4:1101:25527:4554     393     7       144914481       1       88M     =       144914481       0       TTTAGCTCAGTGGTAGAGCGCTTGCCTAGCAAGCGCAAGGCCCTGGGTTCGGTCCTCAGCTCTGGAAAAAAAAAAAAAAAAAAAAAAA   FFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF        AS:i:0  ZS:i:0  XN:i:0  XM:i:0  XO:i:0  XG:i:0  NM:i:0  MD:Z:88 YT:Z:UP RG:Z:geneD_KO1-DD1      NH:i:4
10165:A00977:10:HWCTJDSXX:4:1101:25527:4554     409     8       126503341       1       88M     =       126503341       0       TTTTTTTTTTTTTTTTTTTTTTTCCAGAGCTGAGGACCGAACCCAGGGCCTTGCGCTTGCTAGGCAAGCGCTCTACCACTGAGCTAAA   FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFF        AS:i:0  ZS:i:0  XN:i:0  XM:i:0  XO:i:0  XG:i:0  NM:i:0  MD:Z:88 YT:Z:UP RG:Z:geneD_KO1-DD1      NH:i:4
10166:A00977:10:HWCTJDSXX:4:1101:25527:4554     393     5       12155114        1       88M     =       12155114        0       TTTAGCTCAGTGGTAGAGCGCTTGCCTAGCAAGCGCAAGGCCCTGGGTTCGGTCCTCAGCTCTGGAAAAAAAAAAAAAAAAAAAAAAA   FFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF        AS:i:0  ZS:i:0  XN:i:0  XM:i:0  XO:i:0  XG:i:0  NM:i:0  MD:Z:88 YT:Z:UP RG:Z:geneD_KO1-DD1      NH:i:4
10167:A00977:10:HWCTJDSXX:4:1101:25527:4554     69      13      9054582 0       *       =       9054582 0       CCTTGTTTTTTTTTTGGTCTTTTTGTTATTTTGTCTTTTTTTTTTTTTTTTTTTTTTTTAAAGAGCTTAGGCACGAACACAGGGCCTTG       F,FFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FF::FF:,F,F,,F,::,,:F,FF,,:F,FF,F,::  YT:Z:UP RG:Z:geneD_KO1-DD1

```

### Read pair sample 5

```bash
[chengar2@calculon hisat2]$ grep -in A00977:10:HWCTJDSXX:4:1101:27227:7435 KO1-DD1.pri.hisat2.sam
18179:A00977:10:HWCTJDSXX:4:1101:27227:7435     89      5       31452510        60      68M12874N33M    =       31452510        0       AGCCCTTGGGTGAAATTGTTAGGCGTGGAGGGGGAGTGATGTCTTCCAGACTCGGTGCAGTCACCGCCGGCCCAGTGCTCCATAAAGGATAACAGTTTCCA      FFFFFF:F:FFFFFFFFFFFFFFF:FFFFFFFFFFFFFF:::FFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF   AS:i:-1 XN:i:0  XM:i:0  XO:i:0  XG:i:0  NM:i:0  MD:Z:101        YT:Z:UP RG:Z:geneD_KO1-DD1 XS:A:+  NH:i:1
18180:A00977:10:HWCTJDSXX:4:1101:27227:7435     133     5       31452510        0       *       =       31452510        0       TTGGTCTGTCCGGAGCCCAGGGGTGAAATTGTTAGGCGTGGAGGGGGAGTTATGTCTTCCAGAATCGGTGCAGTAACCCCCGGCCCACTGCTCCATAAA        FF,FFFF:FFF:FF,FF:,,:FFF,FFF,F:FFFF:,:FFFF,FFFFF:F:FF,F,::,,F,F,,:FFFFFF,,,F:,,F:FFFFF:,,:FF,,F,FFF     YT:Z:UP RG:Z:geneD_KO1-DD1
```

Code below is for fixing mate pairs' SAM flag with **`picard FixMateInformation`**. Run **`ValidateSamFile`** again after this is done.

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=picard_fm
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=24gb
#SBATCH --output=/home/chengar2/geneD/picard_fm_%A.out
#SBATCH --error=/home/chengar2/geneD/picard_fm_%A.err


start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load picard/2.13

for sam in /scratch/chengar2/geneD/hisat2/*pri.hisat2.sam 

do
    extract=$(echo $sam | awk -F"." '{print $1"."$2"."$3}')
    name=$(echo $extract | awk -F"/" '{print $NF}')
    echo "running analysis for $name on $(date)"
    java -jar /cm/shared/apps/picard/2.13/picard.jar FixMateInformation I=$sam \
    O=/scratch/chengar2/geneD/hisat2/fixed_mate_$name.sam
    echo "analysis complete for $name on $(date)"
done


end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

### Sorting and Compressing

Reads in SAM files are then sorted by chromosome coordinates and compressed into BAM files.

I used samtools `sort` function to do the job.  
`-T` path to output temporary files and the naming scheme    
`-O` output format; set to `bam` for compression  
`-o` path to output `bam` files and naming scheme  
`-@` no. of threads should be used  

Some program such as `picard`'s `MarkDuplicates` required the `bam` files to be sorted by coordinate. However `featureCounts` required the files to be sorted by queryname (otherwise some files might be mistaken as single-end sequencing). Use `-n` flag if you need to sort by queryname.

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=bam
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=24gb
#SBATCH --output=/home/chengar2/geneD/bam_%A.out
#SBATCH --error=/home/chengar2/geneD/bam_%A.err


start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load samtools/1.9

for sam in /scratch/chengar2/geneD/hisat2/fixed_mate_*pri.hisat2.sam
do
    extract=$(echo $sam | awk -F"." '{print $1"."$2"."$3}')
    name=$(echo $extract | awk -F"/" '{print $NF}')
    echo "running analysis for $name on $(date)"
    samtools sort -T /scratch/chengar2/geneD/bam/$name.sorting.tmp \
    -O bam -o /scratch/chengar2/geneD/bam/$name.sorted.bam -@ 6 $sam
    echo "analysis complete for $name on $(date)"
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds


I ran another round of picard `ValidateSamFile` after the sorting and compression just to be sure.

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=picard
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=24gb
#SBATCH --output=/home/chengar2/geneD/picard_%A.out
#SBATCH --error=/home/chengar2/geneD/picard_%A.err


start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load picard/2.13

for bam in /scratch/chengar2/geneD/bam/*.bam
do
    extract=$(echo $bam | awk -F"." '{print $1}')
    name=$(echo $extract | awk -F"/" '{print $NF}')
    echo "running analysis for $name on $(date)"
    java -jar /cm/shared/apps/picard/2.13/picard.jar ValidateSamFile I=$bam MODE=SUMMARY \
    O=/scratch/chengar2/geneD/bam/sorted_$name.txt
    echo "analysis complete for $name on $(date)"
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds


### Collect Alignment metrics

`I` path to `bam` files  
`O` path to save output `bam` files  
`R` path to reference genome

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=picard-MM
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=24gb
#SBATCH --output=/home/chengar2/geneD/picard-MM_%A.out
#SBATCH --error=/home/chengar2/geneD/picard-MM_%A.err


start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load picard/2.13

for bam in /scratch/chengar2/geneD/bam/*.bam
do
    extract=$(echo $bam | awk -F"." '{print $1"."$2"."$3"."$4}')
    name=$(echo $extract | awk -F"/" '{print $NF}')
    echo "running analysis for $name on $(date)"
    java -jar /cm/shared/apps/picard/2.13/picard.jar CollectMultipleMetrics I=$bam O=$extract.multiple_metrics \
    R=/scratch/chengar2/geneD/dna/Mus_musculus.GRCm38.dna.primary_assembly.fa
    echo "analysis complete for $name on $(date)"
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

### Mark/RMDuplicates

`I` path to `bam` files  
`O` path to save output `bam` files  
`M` path to save metrics  
`REMOVE_DUPLICATES` if set to `true` will remove reads from `bam` entirely; otherwise reads will be marked (0x400 SAM flag will   be added to reads). This can be left out if downstream application can ignore the marked reads

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=mark_dup
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=40gb
#SBATCH --output=/home/chengar2/geneD/dup_%A.out
#SBATCH --error=/home/chengar2/geneD/dup_%A.err


start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load picard/2.13

for bam in /scratch/chengar2/geneD/bam/*.sorted.bam
do
	extract=$(echo $bam | awk -F"." '{print $1"."$2"."$3"."$4}')
	name=$(echo $extract | awk -F"/" '{print $NF}')
	echo "running analysis for $name on $(date)"
	java -jar /cm/shared/apps/picard/2.13/picard.jar MarkDuplicates I=$bam \
	O=/scratch/chengar2/geneD/bam/no-dup_$name.bam M=/scratch/chengar2/geneD/bam/dup-metrics_$name.txt \
	REMOVE_DUPLICATES=true
	echo "analysis complete for $name on $(date)"
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

### Sort by queryname

### Mapping Quality (MAPQ)

I found this ancient blog post about `bowtie2`'s MAPQ: http://biofinysics.blogspot.com/2014/05/how-does-bowtie2-assign-mapq-scores.html

The take home is that lower the MAPQ, lower the confidence in the alignment. This could be due to mismatch between the reads and the reference sequence, a gap, N's penalty, multiple alignment etc. At the end of the day the equation: MAPQ=-10log10(probability of misalignment) stands correct.

Even though `hisat2` is supposedly bulit on `bowtie2`, the MAPQ scoring behavior seems to differ drastically. According to this thread https://www.biostars.org/p/244106/, and also the results of our own `bam` files generated by `hisat2`, there is only 3 possible value of MAPQ: 0, 1, and 60. The comment made by `prasundutta87` in the thread suggests reads with MAPQ of 60 are unqiue reads and anything lower means they have secondary alignment. However `Fabio Marroni` pointed out he has reads with MAPQ of 60 but have multiple alignments, leading us to this thread https://www.biostars.org/p/295354/. `Macspider`'s comment (somewhat) confirms that MAPQ 60 doesn't neccessarily guarantee the read(s) to be unqiuely mapped. 

So all in all filtering out reads that has a MAPQ lower than 2 seems to be the most stringent strategy we can have, which rougly translates to discarding 10-15% of the reads...not a small number but maybe it is worth it for the extra accuracy (?). Now I also wonder whether we should filter out non-primary alignments and/or multiple alignments...things are getting more complicated! But for now I will filter out the "low quality" alignments and leave it as that.

If you need more affirmation to filter reads in `bam` files: https://biostar.usegalaxy.org/p/28856/

Below is the code to filter out reads in the `bam` file with MAPQ <2. Depends on the downstream package used, this might be not necessary.  
`-@` no. of threads  
`-q` is set to 2, the minimum MAPQ we want  
`-b` is used to specify inputs are `bam` files  
`-h` is set to include `bam` header in output

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=filter
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=24gb
#SBATCH --output=/home/chengar2/geneD/filter_%A.out
#SBATCH --error=/home/chengar2/geneD/filter_%A.err

module load samtools/1.9

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

for bam in /scratch/chengar2/geneD/bam/no-dup*
do
    extract=$(echo $bam | awk -F"." '{print $1}')
    name=$(echo $extract | awk -F"_" '{print $NF}')
    echo "running analysis for $name on $(date)"
    samtools view -@ 6 -bhq 2 $bam > /scratch/chengar2/geneD/filtered_bam/filtered_$name.bam
    echo "analysis complete for $name on $(date)"
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

### Index bam

This step is required if you wish to visualize the alignment result in IGV.

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=index_bam
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=24gb
#SBATCH --output=/home/chengar2/geneD/index-bam_%A.out
#SBATCH --error=/home/chengar2/geneD/index-bam_%A.err

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load samtools/1.9

for bam in /scratch/chengar2/geneD/filtered_bam/*.bam
    samtools index $bam
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

### Counting Reads

>Some program such as `picard`'s `MarkDuplicates` required the `bam` files to be sorted by coordinate. However `featureCounts` required the files to be sorted by queryname (otherwise some files might be mistaken as single-end sequencing). Use `-n` flag if you need to sort by queryname.

I have used `summarizeOverlaps` from `GenomicAlignments` to do read-counting before, however the package is not very friendly if we wish to do strand-specific counting, dropping low quality reads, and ignoring duplicated reads all at once. `featureCounts` from `subread` seems to be easier to implement without too much trouble especially it is already installed in our cluster so we can utilize the powerful computer for counting. The authors also released an R version of the program called `Rsubread`. You can run it on R if you want.

Note that read counting is rather convoluted if one wish to only count the exons that code for protein (i.e. ignore reads that aligned to lnc RNA, pseudogenes, retained intron etc.) and would not be reliable without advance modeling in isoform expression. To my understanding, this is because an exon that is transcribed and translated in a protein could also be expressed in a non-protein coding transcript, making the reads that aligned to it rather difficult to categorize without information on individual transcript abundance. This might be worth exploring in the future; for now I would let the program count all read alignments and summarize by "gene_id".


Dr. Friederike Dundar wrote a very detail manual on RNA-seq and **Chapter 4** has extensive details on gene-based read counting vs Isoform counting. Make sure you have a read https://github.com/friedue/course_RNA-seq2017/blob/master/Intro2RNAseq.pdf

Code below is for read-counting with the package `subread`. 

`-T` no. of threads to use  
`-p` is used to specify all files contain pair-end reads  
`-a` the path to the annotation file used (from ensembl; see the `wget` section)  
`-t` count reads aligned to exon...  
`-g` summarized by the meta-feature "gene_id"  
`-Q` is set to the minimum MAPQ score required  
`-s` is set to 2, i.e. library strand = first strand. Program will conduct strand-specific counting and ignore unpaired reads because their strandness cannot be determined  
`--ignoreDup` is used  
`-o` specify output file name or path

Note there is a lot of different approach we can take here. Change the code accordingly to explore different options.
1. To drop duplicate reads or not (~10-15% of alignments)
2. To conduct strand-specific counting or not (some libraries see increased % alignment assignment when strandness is considered due to lower `Unassigned_Ambiguity`)
3. To drop low quality reads or not (~10%-15% of alignments)

There is also options to allow multi-mapping reads to be counted or allow reads to overlap multiple genes in one alignment, you can try them out if you are interested (I don't think they are useful in your case). 

I have also tried changing the `-g` flag to "transcript_id" and "exon", hoping that will allow us to filter out non-protein coding transcripts/exons down the line. Make sure you also use the `-O` flag.

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=picard_fm
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=10
#SBATCH --mem=40gb
#SBATCH --output=/home/chengar2/geneD/picard_fm_%A.out
#SBATCH --error=/home/chengar2/geneD/picard_fm_%A.err

start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load subread/1.6.4

bam=/scratch/chengar2/geneD/bam/md_fm*.qsort.bam
featureCounts -T 10 -p -a /scratch/chengar2/geneD/gtf/Mus_musculus.GRCm38.98.gtf.gz -t exon -g gene_id \
-Q 2 -s 2 --ignoreDup \
-o counts.txt $bam

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds

### Work with the data in R

Once we have the `txt` file from `featureCounts`, we will have to switch over to `R` to continue the analysis.  

This is a very useful link with step-by-step instructions on using `DESeq2`: http://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html

Here's another demo for `DESeq2`: https://www.bioconductor.org/packages/devel/workflows/vignettes/rnaseqGene/inst/doc/rnaseqGene.html

In [ ]:
library(DESeq2)
library(dplyr)
library(tidyr)
setwd("Z:/Arthur/rna_seq_script") 
#or press `ctrl`+`shift`+`H` to change the working directory to where the `txt` count file is located

Below is the comment regarding pre-analysis filtering I left for myself when I was doing the SOX2 revision.

> ### To Filter or Not to Filter  
>A note on filtering raw counts before normalization with DESeq2 or edgeR  
>- The strength of the filter **does** affect the downstream DE genes identification even though DESeq2 and edgeR both have independent filtering to exclude genes with low count from analysis  
>- I think filtering with count-per-million (CPM) is statistically more robust than filtering on the counts directly, as the latter does not account for differences in library sizes between samples.  
>- The simplest form of filter is to sum up the counts (or CPM) for a gene across all samples(libraries) and exclude those that are below a random threshold, e.g. 1, 5, 10  
>- Another way to filter the data is to exlude genes with low expression (again a random threshold) in the majority of libraries (say only 4 out of 5 libraries)  

For the SOX2 revision I took the last approach to filter out genes with low expression in EXCEL. You can open up the `txt` file in EXCEL (because it is tab delimited). First calculate CPM for individual genes (gene_count/total_count_in_library) and then pick a random threshold you like and decide how many libraries is considered "the majority". Once you have done that, exclude genes that fail to meet the threshold, and save the file (as csv, tsv, or txt).
 
Now load it into `R`.

In [ ]:
cts <- read.csv("nd-ss-q2-gene_id.txt",sep="\t", row.names = "Geneid")
head(cts)
colnames(cts)

#the column names are crazy long; simplify them!
colnames(cts)<-sub("X.scratch.chengar2.geneD.bam.md_fm_", "", colnames(cts))
colnames(cts)
colnames(cts)<-sub(".pri.hisat2.qsort.bam", "", colnames(cts))
colnames(cts)

#you don't need the chr, start, end, strand, and length columns. take them out if you haven't done so
#this only contains 16 samples, modify the code after you have the full set of 20
cts<-cts[6:21]
cts

#now make another matrix/dataframe that contains the experimental factors
colnames(cts)
sample<-colnames(cts)
sample

genotype<-c(rep("KO",10), rep("WT", 10))
genotype

trt<-c(rep(c("DD","LP"),10))

#a is KO-DD
#b is KO-LP
#c is WT-DD
#d is WT-LP
group<-c(rep(c("a","b"),5), rep(c("c","d"),5)

#make the data frame
coldata<-data.frame(sample,genotype,trt,group)
coldata
rownames(coldata)<-coldata[,1]
coldata[,1]<-NULL
coldata

#make sure all info are correct in coldata
#make sure all info are correct in coldata
#make sure all info are correct in coldata

#it is also important that all info in coldata is matching the order of the libraries in cts
all(row.names(coldata) %in% colnames(cts))
all(row.names(coldata) == colnames(cts))

#method 1a: the "normal"/"simplest" way
dds <-DESeqDataSetFromMatrix(countData = cts, colData = coldata, design = ~group)
dds
dds <- DESeq(dds)
deg_dd <- results(dds, contrast=c("group","a","c"))
deg_lp <- results(dds, contrast=c("group","b","d"))
ko_induct <- results(dds, contrast=c("group","b","a")) #not a 100% sure if this is right
wt_induct <- results(dds, contrast=c("group","d","c")) #not a 100% sure if this is right
write.csv(deg_dd, "./res/deg_dd.csv") #make sure a folder called "res" exsists in your working dir before you run
write.csv(deg_lp, "./res/deg_lp.csv")
write.csv(wt_induct, "./res/wt_induct.csv")
write.csv(ko_induct, "./res/ko_induct.csv")

#method 1b: stop DESeq2 from changing the padj to NA when read count is low
dds <-DESeqDataSetFromMatrix(countData = cts, colData = coldata, design = ~group)
dds
dds <- DESeq(dds)
deg_dd <- results(dds, contrast=c("group","a","c"), independentFiltering = FALSE)
deg_lp <- results(dds, contrast=c("group","b","d"), independentFiltering = FALSE)
ko_induct <- results(dds, contrast=c("group","b","a"), independentFiltering = FALSE) #not a 100% sure if this is right
wt_induct <- results(dds, contrast=c("group","d","c"), independentFiltering = FALSE) #not a 100% sure if this is right
write.csv(deg_dd, "./nofil/nf_deg_dd.csv") #make sure a folder called "res" exsists in your working dir before you run
write.csv(deg_lp, "./nofil/nf_deg_lp.csv")
write.csv(wt_induct, "./nofil/nf_wt_induct.csv")
write.csv(ko_induct, "./nofil/nf_ko_induct.csv")

#method2: extract the condition/interaction effect
dds <-DESeqDataSetFromMatrix(countData = cts, colData = coldata, design = ~ genotype + trt + genotype:trt)
dds
dds$genotype <- factor(dds$genotype, level=c("WT","KO"))
levels(dds$genotype)
dds <- DESeq(dds)
resultsNames(dds)
res_nofil<-results(dds, name="genotypeKO.trtLP", independentFiltering = FALSE) #toggle "independentFiltering" when necessary
write.csv(res_nofil, file="./res1/nofil.csv")

#method3: LRT test
dds <- DESeqDataSetFromMatrix(countData = cts, colData = coldata, design = ~ genotype + trt + genotype:trt)
dds
dds$genotype <- factor(dds$genotype, level=c("WT","KO"))
levels(dds$genotype)
dds <- DESeq(dds, test="LRT", reduced = ~ genotype + trt)
resultsNames(dds)
lrt<-results(dds, independentFiltering = FALSE)
write.csv(lrt, file="./res1/lrt.csv")    
         

#get the normalized count; this would be useful for plotting graphs
norm_count<-(counts(dds, normalized=TRUE))
write.csv(norm_count, "./res/norm_count.csv")

### Heatmap of DEGs expression

Don't bother with this before the full set of data is acquired; we need to identify the DEGs before we plot the heatmap.

In [ ]:
vsd<-vst(dds,blind=FALSE) #transform the dds using VST

#filter out the nonDEGs
DEG<-read.csv("deseq2_237degs_fdr.05.csv", header=TRUE) #list of ensemblID for the 237 DEGs
filter<-rownames(vsd)%in%DEG$ensemblID
degonly<-vsd[filter,]

#make a matrix of expression minus mean expression
mat<-assay(degonly)-rowMeans(assay(degonly))
anno <- as.data.frame(colData(vsd)[, c("genotype","time_h")])
pheatmap(mat, annotation_col = anno, cluster_cols = FALSE)

#this is borrowed from https://slowkow.com/notes/heatmap-tutorial/ to make each color represent the same number of blocks
quantile_breaks <- function(xs, n = 10) {
breaks <- quantile(xs, probs = seq(0, 1, length.out = n))
breaks[!duplicated(breaks)]
}
mat_breaks <- quantile_breaks(mat, n = 11)
res<-pheatmap(mat, annotation_col = anno, cluster_cols = FALSE, color= viridis(length(mat_breaks) - 1),
breaks= mat_breaks, show_colnames = F, show_rownames = F)

#sorting the dendrogram
library(dendsort)
library(dendextend)
sort_hclust <- function(...) as.hclust(dendsort(as.dendrogram(...)))
mat_cluster_cols <- sort_hclust(res$tree_row)
res<-pheatmap(mat, annotation_col = anno, cluster_cols = FALSE, cluster_rows = rev(mat_cluster_cols), 
              color= viridis(length(mat_breaks)-1), breaks= mat_breaks, show_colnames = F, show_rownames = F, cutree_rows = 9)

### (NOT done yet) Adapter trim only

### Misc

In [ ]:
#extract element(s) from string
ls *txt |awk -F"." '{print $2}'

In [ ]:
#interactive section with cluster
srun --pty -p generalq -t 0-12:00 -c 4 --mem 8G /bin/bash

In [ ]:
#extract MAPQ from sam/bam and export to file
samtools view no-dup_fixed_mate_KO1-DD1.pri.hisat2.sorted.bam | awk -F "\t" '{print $5}' > mapq.txt

In [ ]:
#calculate average MAPQ score
more mapq.txt | awk '{total += $1; count ++ } END {print total/count}'

Below code is for fixing `HEADER_RECORD_MISSING_REQUIRED_TAG` and `MISSING_PLATFORM_VALUE` with Picard `AddOrReplaceReadGroups` function.

In [ ]:
#!/bin/bash

#SBATCH --partition=generalq
#SBATCH --job-name=picard_rg
#SBATCH --mail-type=ALL
#SBATCH --mail-user=
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=24gb
#SBATCH --output=/home/chengar2/geneD/picard_rg_%A.out
#SBATCH --error=/home/chengar2/geneD/picard_rg_%A.err


start=`date +%s`
echo job submitted on $(date) to $HOSTNAME

module load picard/2.13

for sam in /scratch/chengar2/geneD/qual_trim/*.sam
do
    extract=$(echo $sam | awk -F"." '{print $1}')
    name=$(echo $extract | awk -F"/" '{print $NF}')
    echo "running analysis for $name on $(date)"
    java -jar /cm/shared/apps/picard/2.13/picard.jar AddOrReplaceReadGroups I=$sam \
    O=/scratch/chengar2/geneD/qual_trim/rg_$name.sam \
    RGID=geneD_$name \
    RGLB=lib1 \
    RGPL=illumina \
    RGPU=unit1 \
    RGSM=$name \
    VALIDATION_STRINGENCY=SILENT
    echo "analysis complete for $name on $(date)"
done

end=`date +%s`
runtime=$((end-start))
echo job completed on $(date) taking $runtime seconds